In [1]:
# Initial imports
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# from sklearn.metrics import score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [2]:
df = pd.read_csv('etc.csv',parse_dates=True, index_col=('date'))
df.dropna(inplace=True)
df.rename(columns={'date':'Date','PriceUSD':'Close'}, inplace=True)
futures = df.tail(1)

df1 = df[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
futures1 = df1.tail(1)

df2 = df[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
futures2 = df2.tail(1)
df.Close.tail()

date
2019-11-08    4.984974
2019-11-09    4.956965
2019-11-10    5.038860
2019-11-11    4.884171
2019-11-12    4.899531
Name: Close, dtype: float64

In [3]:
df.shape

(841, 37)

In [4]:
# df = df[:-3]
futures.head()

,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMrktCurUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,...,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d
date,,,,,,,,,,,,,,,,,,,,,
2019-11-12,459389,6555,9990787,1524.147521,5.645859e+08,1.257421e+14,0.000172,0.000841,0.000021,0.000103,...,2.515486e+06,18.564551,90.957599,0.965339,4.72971,622599.352604,3.050445e+06,0.045519,0.031577,0.040079


In [5]:
#Model0
X=df[:-1].copy()
X.head()
y = df["Close"][1:].values.reshape(-1, 1)
y[:5]
#Model1
X1=df1[:-1].copy()
X1=X1[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
y1 = df1["Close"][1:].values.reshape(-1, 1)
y1[:1]
#Model2
X2=df2[:-1].copy()
X2=X2[['FeeMedUSD','TxTfrValMedUSD','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
y2 = df2["Close"][1:].values.reshape(-1, 1)
y2[:1]

array([[14.64347978]])

In [6]:
X.tail().shape

(5, 37)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state=78)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=78)

In [8]:
# Scale the training data
scaler = StandardScaler()
scaler1=StandardScaler()
scaler2=StandardScaler()
#Model0
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#model1
X_scaler1 = scaler1.fit(X_train1)
X_train_scaled1 = X_scaler1.transform(X_train1)
X_test_scaled1 = X_scaler1.transform(X_test1)
#model2
X_scaler2 = scaler2.fit(X_train2)
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)

In [57]:
X_train.Close

date
2019-02-27     4.355675
2018-05-20    18.151337
2019-11-11     4.884171
2019-08-28     6.465776
2017-08-12    14.698171
2019-03-07     4.372719
2019-01-26     4.284282
2017-12-08    27.890580
2018-10-25     9.569310
2017-12-10    25.732595
2019-05-27     8.215588
2017-09-09    16.355091
2019-09-12     6.233422
2019-04-03     5.436370
2019-08-27     7.191627
2018-04-20    19.575369
2019-01-21     4.261066
2019-01-19     4.382550
2018-07-24    16.627368
2017-10-08    12.118436
2017-12-02    28.956987
2019-07-27     5.909292
2017-11-02     9.910972
2019-04-12     6.388457
2019-06-24     9.292509
2019-06-10     8.303607
2018-07-28    16.959377
2018-04-16    16.082771
2019-06-16     8.746616
2019-05-15     7.649599
                ...    
2019-10-18     4.427435
2019-03-06     4.328181
2018-09-10    11.296787
2019-02-23     4.819378
2017-12-30    24.737242
2018-09-05    11.969564
2018-11-10     9.397397
2017-10-01    12.811428
2017-09-01    22.400641
2017-07-28    13.918512
2019-10-03 

In [29]:
# Create GradientBoostingClassifier model
#Changed from .75 to .25 11/8
#on 11/8 .75 gave an accuracy score of .929
#model0
model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.25,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model.fit(X_train_scaled,y_train.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model.score(
        X_train_scaled,
        y_train)))
print("Accuracy score (validation): {0:.3f}".format(
    model.score(
        X_test_scaled,
        y_test)))

Accuracy score (training): 1.000
Accuracy score (validation): 0.973


In [30]:
#model1 - [['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
# Create GradientBoostingClassifier model
#Changed from 1 to .05 11/8
#on 11/8 1 gave an accuracy score of .911
model1 = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.05,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model1.fit(X_train_scaled1,y_train1.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model1.score(
        X_train_scaled1,
        y_train1)))
print("Accuracy score (validation): {0:.3f}".format(
    model1.score(
        X_test_scaled1,
        y_test1)))

Accuracy score (training): 0.998
Accuracy score (validation): 0.976


In [31]:
#model2 - [['FeeMedUSD','TxTfrValMedUSD','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
#Changed from .1 to .05 11/8
#on 11/8 .1 gave an accuracy score of .938
model2 = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.05,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model2.fit(X_train_scaled2,y_train2.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model2.score(
        X_train_scaled2,
        y_train2)))
print("Accuracy score (validation): {0:.3f}".format(
    model2.score(
        X_test_scaled2,
        y_test2)))

Accuracy score (training): 0.998
Accuracy score (validation): 0.970


In [32]:
# Make predictions
#model0
predictions = model.predict(X_test_scaled)
y_test = y_test.ravel()
actual = pd.Series(y_test)

#model1
predictions1 = model1.predict(X_test_scaled1)
y_test1 = y_test1.ravel()
actual1 = pd.Series(y_test1)
#model2
predictions2 = model2.predict(X_test_scaled2)
y_test2 = y_test2.ravel()
actual2 = pd.Series(y_test2)

In [33]:
pd.DataFrame({'Model0':predictions,'Model1':predictions1,'Model2':predictions2,'Actual': actual}).head()

,Model0,Model1,Model2,Actual
0,13.685616,14.698044,14.839745,16.541498
1,3.997667,4.111171,4.291850,4.053677
2,14.355324,13.734897,13.629051,14.565106
3,17.652400,18.241251,18.155622,19.896613
4,21.528565,22.013195,22.357296,18.549426


In [34]:
X.tail().shape


(5, 37)

In [35]:
X_future = X_scaler.transform(X.tail(1))
predictions = model.predict(X_future)

X_future1 = X_scaler1.transform(X1.tail(1))
predictions1 = model1.predict(X_future1)

X_future2 = X_scaler2.transform(X2.tail(1))
predictions2 = model2.predict(X_future2)

In [36]:
actual=futures['Close'].ravel()
dates=futures.reset_index()
dates=dates['date'].tail(1).ravel()
#Model1
actual1=futures1['Close'].ravel()
dates1=futures1.reset_index()
dates1=dates1['date'].tail(1).ravel()
#Model2
actual2=futures2['Close'].ravel()
dates2=futures2.reset_index()
dates2=dates2['date'].tail(1).ravel()

In [37]:
test_final=pd.DataFrame({'Model0':predictions,'Model1':predictions1,'Model2':predictions2,'Actual':actual, 'Date':dates})

In [38]:
test_final

,Model0,Model1,Model2,Actual,Date
0,4.90677,4.948824,5.023611,4.899531,2019-11-12


In [53]:
#Model0
real_future = X_scaler.transform(df.tail(1))
predictions = model.predict(real_future)
#Model1
real_future1 = X_scaler1.transform(df1.tail(1))
predictions1 = model1.predict(real_future1)
#Model2
real_future2 = X_scaler2.transform(df2.tail(1).drop('Close',axis=1))
predictions2 = model2.predict(real_future2)

In [54]:
predictions

array([5.02003792])

Based on my predictions today, (Nov 7th)- I should sell my stock tomorrow (Nov 8th)


In [40]:
datelist = pd.date_range((pd.datetime.today()), periods=1).tolist()

# datelist = pd.date_range((pd.datetime.today()+timedelta(days=-3)), periods=5).tolist()
future_dates=[]
for date in datelist:
    future_dates.append(date.strftime("%m-%d-%Y"))

In [41]:
# final=pd.DataFrame({'Date':'11-13','Model0':predictions,'Model1':predictions1,'Model2':predictions2})
final=pd.DataFrame({'Date':pd.datetime.today(),'Model0':predictions,'Model1':predictions1,'Model2':predictions2})


In [42]:
final

,Date,Model0,Model1,Model2
0,2019-11-13 21:11:39.053047,5.020038,4.948824,5.063491


In [43]:
from joblib import dump, load
dump(model, 'etc_model.joblib')

['etc_model.joblib']

In [60]:
model = load('etc_model.joblib')
X_scaler = scaler.fit(df)
real_future = X_scaler.transform(df.tail(1))
predictions = model.predict(real_future)

In [61]:
predictions

array([4.43419529])